In [1]:
import pdb
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import reward as rw
import reward.utils as U

/home/lgvaz/anaconda3/envs/reward/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:
GAMMA = 0.99

In [3]:
use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")

In [4]:
env = rw.env.GymEnv("CartPole-v0")
runner = rw.runner.SingleRunner(env)
batcher = rw.batcher.RolloutBatcher(runner=runner, batch_size=512)

/home/lgvaz/libs/gym/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [5]:
class PolicyNN(nn.Module):
    def __init__(self, num_inputs, num_outputs, activation=nn.Tanh):
        super().__init__()
        self.activation = activation()
        
        self.hidden = nn.Linear(num_inputs, 64)
        self.out = nn.Linear(64, num_outputs)
        
    def forward(self, x):
        x = self.activation(self.hidden(x))
        return self.out(x)  

In [6]:
class CategoricalPolicy(rw.policy.BasePolicy):
    def create_dist(self, s):
        logits = self.nn(s)
        return rw.dist.Categorical(logits=logits)
    
    def get_ac(self, s, step):
        dist = self.create_dist(s)
        return U.to_np(dist.sample())

In [11]:
p_nn = PolicyNN(batcher.s_space.shape[0], 2).to(device)
policy = CategoricalPolicy(p_nn)

p_opt = torch.optim.Adam(p_nn.parameters(), lr=1e-3)

In [13]:
logger = U.Logger('/tmp/logs/cart_pole/pg-v0-3')
last_logged_step = 0

Writing logs to: /tmp/logs/cart_pole/pg-v0-3


In [14]:
for batch in batcher.get_batches(max_steps=3e5, act_fn =policy.get_ac):
    batch = batch.to_tensor()
    # TODO: Calculation is actually a little wrong here, vt_last should not be None
    ret = U.estim.disc_sum_rs(rs=batch.r, ds=batch.d, gamma=GAMMA)
    pdb.set_trace()
    batch = batch.concat_batch()
    
    # Calculate loss
    dist = policy.create_dist(batch.s)
    log_prob = dist.log_prob(batch.ac)
    assert ret.shape == log_prob.shape
    losses = ret * log_prob
    loss = -losses.mean()
    
    # Optimize
    p_opt.zero_grad()
    loss.backward()
    p_opt.step()
    
    # Write logs    
    if batcher.num_steps > last_logged_step:
        last_logged_step = batcher.num_steps + 10000
        batcher.write_logs(logger)
        logger.add_log('policy/loss', loss)
        logger.log(step=batcher.num_steps)    

TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEPRECATED
TO TENSOR DEP

AttributeError: 'list' object has no attribute 'detach'